In [3]:
import pandas as pd
import gzip
import json

Считываем данные. Я выбрала данные по видеоиграм.

In [5]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/content/Video_Games.json.gz')

In [6]:
df

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN
1,3.0,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN
2,4.0,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15,NaN,NaN
3,1.0,True,"02 7, 2014",A2XO1JFCNEYV3T,0439381673,ColoradoPartyof5,The product description should state this clea...,does not work on Mac OSX,1391731200,11,NaN,NaN
4,4.0,True,"01 16, 2014",A19WLPIRHD15TH,0439381673,Karen Robinson,I would recommend this learning game for anyon...,Roughing it,1389830400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2565344,5.0,True,"08 1, 2018",ANGB54K3888S4,B01HJEBIAA,josh,"Love it, work good",Works good,1533081600,NaN,NaN,NaN
2565345,5.0,True,"07 17, 2018",A3TEVKR0ZVQB2T,B01HJEBIAA,Prime Member,I do a lot of copy/paste and other keyboard sh...,Great mouse for work and gaming,1531785600,NaN,NaN,[https://images-na.ssl-images-amazon.com/image...
2565346,5.0,True,"07 6, 2018",ABE7YPWEHNVJZ,B01HJEBIAA,Amazon Customer,One year in and it's still working great! Hig...,Five Stars,1530835200,NaN,NaN,NaN
2565347,5.0,True,"06 12, 2018",A3ES9QBK3G192O,B01HJEBIAA,Lina Marmolejos,EXCELENTE,Five Stars,1528761600,NaN,NaN,NaN


In [7]:
df = df[["asin", "reviewText"]]
df = df.dropna()

1.  3 способа найти упоминания товара в отзывах:



1.   Сделать что-то вроде семантической карты: для основного названия товара найти ближайших соседей по косинусной близости и из коллокаций с ними, возможно, получится что-то вытащить.
2.   Снова залезть в семантические структуры: с помощью ворднета можно вытаскивать гипо/ гиперонимические отношения и им подобные. В конце получится некоторый список слов, состоящих в семантических отношениях с основным названием товара и можно будет искать в отзывах n-граммы с этими словами.
3. Написать правила. Кроме natasha|yargy можно использовать spacy, в нем есть EntityRecognizer, по крайней мере для английского, с помощью правил можно будет повысить точность. Сложность в том, что названия товаров - скорее всего, не будут стандартными сущностями..


2. Реализовать один из способов (пытаюсь написать правила. Т к у нас видеоигры - там упоминаниями товара скорее всего будут или существительные, или наиболее частотные н-граммы - у игр часто многословные названия)

In [8]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 8.2 MB 42.6 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=190b5a8931316e1657fa6efbb76ad2b1567d3507938c1b92748cf60a45fbd9b2
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [12]:
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import string
nltk.download('omw-1.4')
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')

lst = df['reviewText']. tolist ()
text = ' '.join(lst)


tokens = [w.lower().strip(string.punctuation) for w in word_tokenize(text) if w.isalpha()]

lemmas = []
for i in range(0, len(tokens)):
    word = morph.parse(tokens[i])[0].normal_form
    lemmas.append(word)
lemmas = [w for w in lemmas if w not in sw]

bigrams = nltk.bigrams(lemmas)
frequency1 = nltk.FreqDist(bigrams)
frequency1 = dict(sorted(frequency1.items(), key=lambda kv: kv[1], reverse=True))
k = 0
print('Биграммы:')
for key,value in frequency1.items():
    if k <= 20:
        print(key, value)
        k += 1
    else:
        break
    
trigrams = nltk.trigrams(lemmas)
frequency2 = nltk.FreqDist(trigrams)
frequency2 = dict(sorted(frequency2.items(), key=lambda kv: kv[1], reverse=True))
n = 0
print('Триграммы:')
for key,value in frequency2.items():
    if n <= 20:
        print(key, value)
        n += 1
    else:
        break



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: ignored